In [10]:
import numpy as np
import pandas as pd
import requests
import xarray as xr
import matplotlib.pyplot as plt
from climate_indices import compute, indices


# Define Lankien coordinates
latitude = 8.4281
longitude = 33.7833

# Define time range
start_date = "2015-01-01"
end_date = "2025-03-04"

# Function to fetch NASA POWER data
def fetch_nasa_power(lat, lon, start, end, params=["PRECTOT", "PET"]):
    url = f"https://power.larc.nasa.gov/api/temporal/monthly/point?parameters={','.join(params)}&community=SB&longitude={lon}&latitude={lat}&start={start}&end={end}&format=JSON"
    response = requests.get(url)
    data = response.json()
    if "parameters" in data:
        df = pd.DataFrame(data['parameters']).T
        df.index = pd.to_datetime(df.index, format='%Y%m')
        return df
    else:
        print("Error fetching NASA POWER data:", data)
        return None

# Fetch precipitation and PET data
nasa_data = fetch_nasa_power(latitude, longitude, "201501", "202502")
if nasa_data is not None:
    precipitation = nasa_data["PRECTOT"]
    pet = nasa_data["PET"]
    
    # Calculate SPEI using precipitation and PET
    clim_data = precipitation - pet
    spei_values = compute.spei(series=clim_data, scale=12)
    
    # Plot SPEI
    plt.figure(figsize=(12, 5))
    plt.plot(clim_data.index, spei_values, label="Local SPEI (NASA POWER)", color='blue')
    plt.axhline(y=0, color='black', linestyle='--')
    plt.title("SPEI for Lankien (2015-2025)")
    plt.xlabel("Year")
    plt.ylabel("SPEI")
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("Failed to retrieve data.")


ModuleNotFoundError: No module named 'distutils'